# Carbon dioxide solubility in the NaCl-brine

This tutorial demonstrates how to simulate the solubility of CO<sub>2</sub> gas in the NaCl-brine and its dependence on
the salinity of the brine (also referred to as a salting-out effect).

The chemical system and the corresponding equilibrium solver are initialized below.

In [35]:
from reaktoro import *

# Initialize a thermodynamic database
db = PhreeqcDatabase("phreeqc.dat")

# Create an aqueous phase automatically selecting all species with provided elements
aqueousphase = AqueousPhase(speciate("H O C Na Cl"))
aqueousphase.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2"),
))

# Create a gaseous phase
gaseousphase = GaseousPhase("CO2(g)")
gaseousphase.setActivityModel(ActivityModelPengRobinson())

phases = Phases(db)
phases.add(aqueousphase)
phases.add(gaseousphase)

# Create the chemical system
system = ChemicalSystem(phases)

# Create the equilibrium solver
solver = EquilibriumSolver(system)

Finally, we define the range of temperatures and pressure for the equilibrium calculations and calculate CO<sub>2</sub>
solubilities for different temperatures and brine concentrations.

We start the definition of function `solubility_co2()` that calculates the solubility of CO<sub>2</sub> in the NaCl-brine
by returning the concentration of carbon that got dissolved in the brine.

In [36]:

def solubility_co2(system, solver, T, P, mNaCl):

    # Initial amount of the CO2 gas
    n0CO2g = 10.0

    # Define initial chemical state corresponding to the NaCl-brine of the given concentration
    state = ChemicalState(system)
    state.setTemperature(T, "celsius")
    state.setPressure(P, "bar")
    state.set("H2O"   , 1.0   , "kg")
    state.set("CO2(g)", n0CO2g, "mol")
    state.set("Na+"   , mNaCl , "mol")
    state.set("Cl-"   , mNaCl , "mol")

    # Calculate equilibrium state
    res = solver.solve(state)

    # Throw exception if the equilibrium couldn't be found
    if not res.optima.succeeded:
        raise RuntimeError("Equilibrium calculation did not succeed!")

    # Fetch resulting aqueous properties of the chemical state
    aqprops = AqueousProps(state)

    # Return concentration of the carbon in the aqueous phase
    return aqprops.elementMolality("C")[0]

Finally, we define the range of temperatures and pressure for the equilibrium calculations and calculate CO<sub>2</sub>
solubilities. In a loop, we run equilibrium calculations for different input temperatures and different brine concentrations.

In [37]:
import numpy as np
import pandas as pd

temperatures = np.arange(25.0, 90.0, 5.0)
molsNaCl  = np.array([1.0, 2.0, 4.0])
P = 100.0

df = pd.DataFrame(columns=["T", "amountNaCl", "amountCaq"])

for molNaCl in molsNaCl:
    for T in temperatures:

        # Initial amount of the CO2 gas
        n0CO2g = 10.0

        # Define initial chemical state corresponding to the NaCl-brine of the given concentration
        state = ChemicalState(system)
        state.setTemperature(T, "celsius")
        state.setPressure(P, "bar")
        state.set("H2O"   , 1.0   , "kg")
        state.set("CO2(g)", n0CO2g, "mol")
        state.set("Na+"   , molNaCl , "mol")
        state.set("Cl-"   , molNaCl , "mol")

        # Calculate equilibrium state
        res = solver.solve(state)

        # Throw exception if the equilibrium couldn't be found
        if not res.optima.succeeded: continue

        # Fetch resulting aqueous properties of the chemical state
        aqprops = AqueousProps(state)

        # Update value ["T", "amountNaCl", "amountCaq"] in the dataframe
        df.loc[len(df)] = [T, molNaCl, float(aqprops.elementMolality("C"))]

To visualize the obtained results, we use [bokeh](https://bokeh.org/) plotting library.

In [38]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

Below, we plot the solubility of CO<sub>2</sub>(g) as a function of pressure for different salinities of NaCl-brine.

In [39]:
hovertool = HoverTool()
hovertool.tooltips = [("T", "@T"),
                      ("amount(Ca) in brine", "@amountCaq mol")]

p = figure(
    title="SOLUBILITY OF CO2 IN NACL BRINE",
    x_axis_label=r'TEMPERATURES [°C]',
    y_axis_label='AMOUNT OF DISSOLVED CO2 [MOL/KGW]',
    sizing_mode="scale_width",
    plot_height=300)

p.add_tools(hovertool)

colors = ['teal', 'darkred', 'indigo', 'coral']
for molNaCl, color in zip(molsNaCl, colors):
    df_NaCl = ColumnDataSource(df[df['amountNaCl'] == molNaCl])
    p.line("T", "amountCaq", legend_label=f'{molNaCl} mol of NaCl', line_width=3, line_cap="round", line_color=color, source=df_NaCl)

p.legend.location = 'top_right'
show(p)

As expected, the CO<sub>2</sub>(g)-solubility decreases with the increase of the temperature. Moreover, we see that
the solubility of the gas drops as the brine becomes more saline (also referred as salting out effect).